In [13]:
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
from config import *
from functions import *

In [14]:
# Getting base URL from API Service and API KEY
url = 'http://dataservice.accuweather.com/currentconditions/v1/topcities/150?apikey='
api_key = api_key

# Building full URL with API KEY
full_url = url + api_key

response = requests.get(full_url).json()

# Initialize lists to store the data obtained from the API

countries = []
cities = []
description = []
temperature = []
measure_date = []
datetimes = []

# Iterate over each element of the obtained JSON
for data in response:
    # Extract relevant data
    country_name = data['Country']['LocalizedName']
    city_name = data['LocalizedName']
    weather_text = data['WeatherText']
    temperature_C = data['Temperature']['Metric']['Value']
    timestamp = data['LocalObservationDateTime']
    datetime_insertion = datetime.now()

    # Append data to lists
    countries.append(country_name)
    cities.append(city_name)
    description.append(weather_text)
    temperature.append(temperature_C)
    measure_date.append(timestamp)

    # Create DataFrame
    df = pd.DataFrame({
        "Country Name": countries,
        "City Name": cities,
        "Description":description,
        "Temperature (C)": temperature,
        "Measure date": measure_date,
        "Insertion date": datetime_insertion
    })

In [16]:
# Create RedShift connection
redshift_url = redshift_host
redshift_database = redshift_database
redshift_user = redshift_user
redshift_pwd = redshift_pwd
redshift_port = redshift_port
conn = connect_to_redshift(redshift_url, redshift_database, redshift_user, redshift_pwd, redshift_port)

NameError: name 'redshift_port' is not defined

In [ ]:
# Create table if not exist

with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS marianolicera3_coderhouse.weather
        (
            nombre_pais VARCHAR(255),
            nombre_ciudad VARCHAR(255),
            descripcion VARCHAR(50),
            temperatura FLOAT,
            fecha_medicion DATETIME,
            fecha_insercion DATETIME 
        )
    """)
    conn.commit()

In [ ]:
# Insert data into RedShift table
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO weather (nombre_pais, nombre_ciudad, descripcion, temperatura, fecha_medicion, fecha_insercion)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()